In [113]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib 

<module 'MatchingAlgorithm' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\MatchingAlgorithm.py'>

In [27]:
import MatchingAlgorithm 
importlib.reload(MatchingAlgorithm)

<module 'MatchingAlgorithm' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\MatchingAlgorithm.py'>

In [111]:
import FeatureEngineering
importlib.reload(FeatureEngineering)


<module 'FeatureEngineering' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\FeatureEngineering.py'>

### Setting up

In [3]:
# INPUT YOUR API KEY

API_key = ''

In [28]:
#Reading the Simulated data we have generated previously. 
orders_df = pd.read_csv("Simulated Data/order_data_1576.csv")
catchments_df = pd.read_csv("Simulated Data/catchment_data_1576.csv")
phlebs_df = pd.read_csv("Simulated Data/phleb_data_1576.csv")

In [30]:
#orders_df = orders_df.iloc[:15]  #Uncomment to limit the rows
#phlebs_df = phlebs_df.iloc[:2]   #Uncomment to limit the rows
catchments_df = catchments_df.iloc[:1] #remove this for multi-ends 

In [16]:
#Uncomment below if you are NOT using the Simulated data above
#coordinate_list = FeatureEngineering.get_coordinates_list(orders_df, catchments_df, phlebs_df)
#time_matrix =  FeatureEngineering.create_time_matrix(coordinate_list, API_key)

time_matrix = np.loadtxt('Simulated Data/timeMatrix.txt') #time matrix generated previously, to save API cost

### Running algorithm

In [31]:
# Alternative method of running the Matching Algorithm:
# output = MatchingAlgorithm.run_algorithm(orders_df, catchments_df, phlebs_df, API_key)
# print(output)

output = MatchingAlgorithm.run_algorithm_version_timeMatrix(orders_df, catchments_df, phlebs_df, time_matrix)
print(output)

{
  "Metadata": {
    "Locations": [
      {
        "Location Index": 0,
        "Coordinate": "28.453012121886808,77.04885293025431",
        "Order Id": "Ending Location"
      },
      {
        "Location Index": 1,
        "Coordinate": "28.446275918004297,77.05666911676877",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 2,
        "Coordinate": "28.436374190228392,77.10818368839077",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 3,
        "Coordinate": "28.46043105269203,77.12523221919234",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 4,
        "Coordinate": "28.44673371496172,77.08386447523183",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 5,
        "Coordinate": "28.450267421182843,77.0411748655223",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 6,
        "Coordinate": "28.448895764608586,7

### Reverse Algorithm

In [26]:
# For example, there is a new customer who want a Regular service at A location (28.44004408,77.09423916). This regular service requires 5 minutes of servicing time. 
# This customer wants to view available time slot for the service.
# Below is how we use Reverse Objective function to achieve that:

vacancies = MatchingAlgorithm.reverse_getVacancy_algorithm("28.44004408,77.09423916", 5, ['expertise_regular'], output, API_key)
pd.DataFrame(json.loads(vacancies))

,PhlebotomistIndex,TotalTravelTime,TimeWindowStart,TimeWindowEnd,FromLocIdx,ToLocIdx,FromLocCoordinates,ToLocCoordinates
0,33,26,7,8,79,99,"28.443509544075194,77.07793040413067","28.453049410734078,77.0609194240715"


### Prescriptive Analytics

In [7]:
phlebs_df['service_rating'] = 5 #ensure all service rating is the same

In [56]:
demand_ratios = [(1, 0, 0), (0.8, 0.2, 0), (0.6, 0.2, 0.2),  (0.4, 0.4, 0.2), (0.2, 0.6, 0.2), (0, 0.6, 0.4), (0, 0.8, 0.2), (0, 1, 0),
                 (0, 0.5, 0.5), (0, 0.4, 0.6), (0, 0.2, 0.8), (0, 0, 1)]

full_ratios = []
full_numbers = []
full_costs = []
full_times = []

for demand in demand_ratios:
    list_of_ratios, list_of_total_transit_time, list_of_total_cost, list_of_numbers, list_of_jsons, list_of_chosen_phlebs = MatchingAlgorithm.run_prescriptive_analysis(
        orders_df, catchments_df, phlebs_df, time_matrix, demand[0], demand[1], demand[2])
    full_ratios.append(list_of_ratios)
    full_numbers.append(list_of_numbers)
    full_costs.append(list_of_total_cost)
    full_times.append(list_of_total_transit_time)

In [57]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

mean_ratios = [np.mean(list_of_ratios, axis=0) for list_of_ratios in full_ratios]

fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[0] for ratio in mean_ratios],
    name="Regular" 
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[1] for ratio in mean_ratios],
    name="Premium" 
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[2] for ratio in mean_ratios],
    name="Special" 
), row=3, col=1)


fig.update_layout(height=800, width=1300, title_text="Demand Ratios over Supply Ratios")
fig.show()

### Visualisation (Optional)

In [50]:
# import pygeos
import RouteVisualisation
import geopandas as gpd

importlib.reload(RouteVisualisation)

gpd.options.use_pygeos = False

In [51]:
addresses_list = FeatureEngineering.get_coordinates_list(orders_df, catchments_df, phlebs_df)

In [ ]:
json_result = json.loads(list_of_jsons[1])

polygon = gpd.read_file("Simulation\Gurugram_sample_Polygon.geojson")
addresses_list = addresses_list

RouteVisualisation.visualise_routes(json_result, polygon)